In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import xarray as xr
from scipy.stats import zscore

In [4]:
from neurodsp.plts.time_series import plot_time_series
from ripple_detection.core import gaussian_smooth, segment_boolean_series

In [5]:
from ecephys.data import paths
from ecephys.utils import load_df_h5, add_attrs, get_disjoint_interval_intersections, get_interval_complements
from ecephys.scoring import get_separated_wake_hypnogram, write_visbrain_hypnogram
import ecephys.plot as eplt

In [6]:
SUBJECT = "Doppio"
CONDITION = "REC-6-4"
RECORDING_LENGTH = 7200

In [7]:
lfp_bandpower = xr.open_dataset(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lfp_bandpower.nc"))
emg = xr.open_dataset(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="emg.nc")).emg

In [8]:
def my_gaussian_smooth(da, smoothing_sigma=10):
    estimated_fs = 1 / np.diff(da.time.values).mean()
    return gaussian_smooth(da, smoothing_sigma, estimated_fs)

In [9]:
def below_threshold_intervals(sig, threshold, time):
    _is_below_threshold = pd.Series(sig < threshold, index=time)
    return segment_boolean_series(_is_below_threshold, minimum_duration=0)

def above_threshold_intervals(sig, threshold, time):
    _is_below_threshold = pd.Series(sig > threshold, index=time)
    return segment_boolean_series(_is_below_threshold, minimum_duration=0)

In [10]:
SMOOTHING_SIGMA = 4
TD_THRESHOLD_ZSCORE = 1
EMG_THRESHOLD_ZSCORE = 0
MINIMUM_QWK_DURATION = 10

In [11]:
td_smooth = my_gaussian_smooth(lfp_bandpower.sr_theta / lfp_bandpower.sr_delta, smoothing_sigma=SMOOTHING_SIGMA)
tds_threshold = td_smooth[zscore(np.log(td_smooth)) < TD_THRESHOLD_ZSCORE].max()
tds_below_threshold_intervals = below_threshold_intervals(td_smooth, tds_threshold, lfp_bandpower.time.values)

In [12]:
emg_smooth = my_gaussian_smooth(emg, smoothing_sigma=SMOOTHING_SIGMA)
emgs_threshold = emg_smooth[zscore(emg_smooth) < EMG_THRESHOLD_ZSCORE].max()
emgs_below_threshold_intervals = below_threshold_intervals(emg_smooth, emgs_threshold, emg.time.values)

In [13]:
qwk_intervals = np.asarray(get_disjoint_interval_intersections(emgs_below_threshold_intervals, tds_below_threshold_intervals))
qwk_durations = np.asarray([end - start for start, end in qwk_intervals])
qwk_intervals = qwk_intervals[qwk_durations > MINIMUM_QWK_DURATION]
awk_intervals = get_interval_complements(qwk_intervals, 0, RECORDING_LENGTH)

In [14]:
hypnogram = get_separated_wake_hypnogram(qwk_intervals, awk_intervals)

In [15]:
write_visbrain_hypnogram(hypnogram, paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="hypnogram.txt"))

In [16]:
fig, axes = plt.subplots(2, 1, figsize=(20, 4))
plot_time_series(lfp_bandpower.time.values, td_smooth, ax=axes[0], title="Theta / Delta ratio, smoothed.", ylabel="Ratio")
plot_time_series(emg.time.values, emg_smooth, ax=axes[1], title="Buzsaki-style EMG, smoothed", ylabel="Corr")
for i in range(0, len(axes)):
    eplt.plot_hypnogram_overlay(hypnogram, ax=axes[i])
    axes[i].set_xlim(axes[0].get_xlim())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
lfp_bandpower.close()
emg.close()